# 1. Energy data

# M = 80

## 1.0 Startup

In [1]:
disp ('executing gpml startup script...')
mydir = fileparts (mfilename ('fullpath'));                 % where am I located
addpath (mydir)
dirs = {'cov','doc','inf','lik','mean','prior','util'};           % core folders
for d = dirs, addpath (fullfile (mydir, d{1})), end
dirs = {{'util','minfunc'},{'util','minfunc','compiled'}};     % minfunc folders
for d = dirs, addpath (fullfile (mydir, d{1}{:})), end
% addpath([mydir,'/util/sparseinv'])
pkg load statistics

executing gpml startup script...


In [2]:
dataset = 'energy_data'
load ('./data/energy_data.mat');
[ndata, ~] = size(M);
X = M(:,[1:25,27:28]);
Y = M(:,26);
R = randperm(ndata);        
xt = X(R(1:1735),:);
yt = Y(R(1:1735));
R(1:1735) = [];
x = X(R,:);          
y = Y(R);
n = 18000; nt = 1735;   %Ms = 15

dataset = energy_data


In [3]:
sf2 = 1 ; ell = 1 ; sn2 = 0.1 ; 
d = size(x,2);
hyp.cov = log([ones(d,1)*ell;sf2]); hyp.lik = log(sn2); hyp.mean = [];
opts.Xnorm = 'Y' ; opts.Ynorm = 'Y' ;
opts.Ms = 80 ;
opts.ell = ell ; opts.sf2 = sf2 ; opts.sn2 = sn2 ;
opts.meanfunc = []; opts.covfunc = @covSEard; opts.likfunc = @likGauss; opts.inffunc = @infGaussLik ;

partitionCriterion = 'kmeans'; %, 'kmeans', 'knkmeans'
opts.numOptFC = 25 ;
opts.partitionCriterion = partitionCriterion ;
[models,t_dGP_train] = aggregation_train(x,y,opts) ;
save('./res/energymodel80', 'models','t_dGP_train')

In [4]:
partitionCriterion = 'kmeans'; %, 'kmeans', 'knkmeans'
opts.numOptFC = 25 ;
opts.partitionCriterion = partitionCriterion ;
s = load('./res/energymodel80');
models = s.models;


## 1.1 RBCM vs TERBCM

### 1.1.1 RBCM baseline

In [5]:
criterions = {'RBCM'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing energy_data kmeans RBCM
RBCM: SMSE 0.0015, MSLL 5.0504, NLPD 7.9363


### 1.1.2 Grid q

In [6]:
partitionCriterions = {'kmeans'}; % 'random', 'kmeans', 'knkmeans'
criterions = {'TERBCM'}; %, 'TEGRBCM' 'TEGPoE'};
qs = 0.2:0.2:3.0;

for i = 1:length(partitionCriterions)
    partitionCriterion = partitionCriterions{i};
    for j = 1:length(criterions)     
        criterion = criterions{j};
        printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
        opts.partitionCriterion = partitionCriterion ;
        %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
        for q = qs
            [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
            [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
            printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f - q %6.2f\r\n', criterion, SMSE,MSLL,NLPD,q);
            filename = sprintf('./res/terbcm%s_%s_%s_%6.2f.mat', dataset, partitionCriterion, criterion, q);
            save(filename, 'SMSE','MSLL','NLPD')
        endfor
    endfor
endfor


Processing energy_data kmeans TERBCM
TERBCM: SMSE 0.0015, MSLL 1.1682, NLPD 4.0540 - q   0.20
TERBCM: SMSE 0.0015, MSLL 0.1437, NLPD 3.0296 - q   0.40
TERBCM: SMSE 0.0015, MSLL 0.4806, NLPD 3.3665 - q   0.60
TERBCM: SMSE 0.0015, MSLL 1.8923, NLPD 4.7781 - q   0.80
TERBCM: SMSE 0.0015, MSLL 5.0504, NLPD 7.9363 - q   1.00
TERBCM: SMSE 0.0015, MSLL 11.8144, NLPD 14.7002 - q   1.20
TERBCM: SMSE 0.0016, MSLL 26.7838, NLPD 29.6696 - q   1.40
TERBCM: SMSE 0.0016, MSLL 61.5848, NLPD 64.4706 - q   1.60
TERBCM: SMSE 0.0016, MSLL 146.6757, NLPD 149.5615 - q   1.80
TERBCM: SMSE 0.0016, MSLL 364.7270, NLPD 367.6129 - q   2.00
TERBCM: SMSE 0.0016, MSLL 947.6414, NLPD 950.5273 - q   2.20
TERBCM: SMSE 0.0016, MSLL 2566.1508, NLPD 2569.0367 - q   2.40
TERBCM: SMSE 0.0016, MSLL 7216.7504, NLPD 7219.6362 - q   2.60
TERBCM: SMSE 0.0017, MSLL 21006.4530, NLPD 21009.3388 - q   2.80
TERBCM: SMSE 0.0017, MSLL 63108.9870, NLPD 63111.8728 - q   3.00


In [35]:
partitionCriterions = {'kmeans'}; % 'random', 'kmeans', 'knkmeans'
criterions = {'TERBCM'}; %, 'TEGRBCM' 'TEGPoE'};
qs = -0.2:-0.2:-3.0;

for i = 1:length(partitionCriterions)
    partitionCriterion = partitionCriterions{i};
    for j = 1:length(criterions)     
        criterion = criterions{j};
        printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
        opts.partitionCriterion = partitionCriterion ;
        %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
        for q = qs
            [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
            [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
            printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f - q %6.2f\r\n', criterion, SMSE,MSLL,NLPD,q);
            filename = sprintf('./res/terbcm%s_%s_%s_%6.2f.mat', dataset, partitionCriterion, criterion, q);
            save(filename, 'SMSE','MSLL','NLPD')
        endfor
    endfor
endfor


Processing energy_data kmeans TERBCM
TERBCM: SMSE 0.0015, MSLL -6.4023, NLPD -3.5165 - q  -0.20
TERBCM: SMSE 0.0015, MSLL -6.1899, NLPD -3.3040 - q  -0.40
TERBCM: SMSE 0.0015, MSLL -6.0584, NLPD -3.1726 - q  -0.60
TERBCM: SMSE 0.0015, MSLL -5.9636, NLPD -3.0778 - q  -0.80
TERBCM: SMSE 0.0015, MSLL -5.8909, NLPD -3.0051 - q  -1.00
TERBCM: SMSE 0.0015, MSLL -5.8332, NLPD -2.9473 - q  -1.20
TERBCM: SMSE 0.0015, MSLL -5.7865, NLPD -2.9007 - q  -1.40
TERBCM: SMSE 0.0015, MSLL -5.7484, NLPD -2.8625 - q  -1.60
TERBCM: SMSE 0.0015, MSLL -5.7171, NLPD -2.8313 - q  -1.80
TERBCM: SMSE 0.0015, MSLL -5.6915, NLPD -2.8056 - q  -2.00
TERBCM: SMSE 0.0015, MSLL -5.6705, NLPD -2.7847 - q  -2.20
TERBCM: SMSE 0.0015, MSLL -5.6536, NLPD -2.7677 - q  -2.40
TERBCM: SMSE 0.0015, MSLL -5.6401, NLPD -2.7542 - q  -2.60
TERBCM: SMSE 0.0015, MSLL -5.6296, NLPD -2.7438 - q  -2.80
TERBCM: SMSE 0.0015, MSLL -5.6218, NLPD -2.7359 - q  -3.00


### 1.1.3 Find single q

In [5]:
criterion = 'TERBCM'; 

In [37]:
printf('Optimizing q\r\n');
q = grad_q(models, criterion, opts, iter=50, bs=100, lr=0.001, lambda=10, init_q=1.5);

Optimizing q
Optimizing q: remain iter - 50, q - 1.5000
grad_q_norm = -16.968
grad_q_reg =  0.41166
Optimizing q: remain iter - 49, q - 1.5000
grad_q_norm = -92.479
grad_q_reg =  0.23042
Optimizing q: remain iter - 48, q - 1.5000
grad_q_norm =   -4.6171e+04
grad_q_reg = -115.06
Optimizing q: remain iter - 47, q - 1.4940
grad_q_norm =   -4.5385e+04
grad_q_reg = -25.463
Optimizing q: remain iter - 46, q - 1.4882
grad_q_norm =   -5.3197e+04
grad_q_reg = -171.90
Optimizing q: remain iter - 45, q - 1.4813
grad_q_norm = -8098.4
grad_q_reg = -120.42
Optimizing q: remain iter - 44, q - 1.4801
grad_q_norm =   -5.6231e+06
grad_q_reg = -1398.6
Optimizing q: remain iter - 43, q - 0.7665
grad_q_norm =   -1.1014e+05
grad_q_reg =  18.319
Optimizing q: remain iter - 42, q - 0.7526
grad_q_norm = -414.11
grad_q_reg =  10.521
Optimizing q: remain iter - 41, q - 0.7525
grad_q_norm = -710.24
grad_q_reg =  24.907
Optimizing q: remain iter - 40, q - 0.7525
grad_q_norm =   -1.6077e+04
grad_q_reg =  9.7758
Opt

In [34]:
[mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

### 1.1.4 Find multi q

In [8]:
printf('Optimizing q\r\n');
iqs = 1.01 + rand(opts.Ms,1);
qs = mul_grad_q(models, criterion, opts, iter=50, bs=100, lr=0.2, lambda=10, init_q=iqs);
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);

Optimizing q
Optimizing qs: remain iter - 50
qs =

   1.2982
   1.4277
   1.4980
   1.6509
   1.5328
   1.9616
   1.2746
   1.8603
   1.7628
   1.6598
   1.4319
   1.8357
   1.2009
   1.6096
   1.4868
   1.8512
   1.3512
   1.8016
   1.7612
   1.3015
   1.7219
   1.6004
   1.6881
   1.8780
   1.6427
   1.3613
   1.3710
   2.0069
   1.6889
   1.3322
   1.8115
   1.3545
   1.2152
   1.4076
   1.9678
   1.9935
   1.1174
   1.7557
   1.1635
   1.7970
   1.8382
   1.9039
   1.3435
   1.2226
   1.4644
   1.5814
   1.9234
   1.0832
   1.2919
   1.9213
   1.8813
   1.5003
   1.9398
   1.6942
   1.8083
   1.5006
   1.4736
   1.4957
   1.1288
   1.4580
   1.8320
   1.1095
   1.7843
   1.6667
   1.3955
   1.2055
   1.0911
   1.3900
   1.7309
   1.1802
   1.7046
   1.4394
   1.4025
   1.8112
   1.9750
   1.4321
   1.6594
   1.9514
   1.3828
   1.0984

ttb =    3.5575e+07
Optimizing qs: remain iter - 49
qs =

   1.2980
   1.4278
   1.4972
   1.6505
   1.5334
   1.9616
   1.2742
   1.8602
   1.7628


   1.30090
   1.41396
   1.49643
   1.65018
   1.52854
   1.95453
   1.27318
   1.05884
   1.76284
   1.65976
   1.43148
   1.80184
   1.07024
   1.60928
   1.48926
   1.85124
   1.35131
   1.78536
   1.76123
   0.50294
   1.72186
   1.59984
   1.69114
   1.87720
   1.64220
   1.36109
   1.36977
   1.20924
   1.68989
   1.33261
   1.81244
   1.35465
   1.21816
   1.40722
   1.96784
   1.99322
   1.10033
   1.75366
   1.10932
   1.79698
   1.83801
   1.90373
   1.34150
   1.22203
   1.24911
   1.58035
   1.92343
   1.08508
   1.29358
   1.92101
   1.88081
   0.74880
   1.94205
   1.69425
   1.80791
   1.50053
   1.47730
   1.49467
   1.13111
   1.45795
   1.81194
   1.11223
   1.78433
   1.66709
   1.39603
   1.20558
   1.09120
   1.38993
   1.73252
   1.18011
   1.70610
   1.43926
   1.40196
   1.79227
   1.97486
   1.43217
   1.65630
   1.95162
   0.58239
   1.09948

ttb =    6.2126e+05
Optimizing qs: remain iter - 40
qs =

   1.30106
   1.41409
   1.49686
   1.65018
   1.52714
   1.9

   1.11609
   0.98311
   1.66666
   1.39364
   1.10750
   1.09168
   1.36606
   1.73297
   1.18172
   1.70478
   1.43844
   1.40251
   1.79375
   1.97346
   1.43290
   1.65625
   1.95175
   0.60694
   0.92008

ttb =    4.5878e+05
Optimizing qs: remain iter - 32
qs =

   1.12412
   1.41376
   1.49674
   1.64980
   1.52758
   1.95369
   1.27255
   1.06047
   1.76302
   1.65972
   1.43008
   1.00242
   1.06762
   1.60826
   1.48902
   1.85124
   1.35191
   1.78385
   1.74830
   0.50849
   1.72188
   1.60151
   1.69202
   1.87694
   1.64283
   1.36081
   1.35867
   1.20880
   1.68852
   1.33227
   1.81212
   1.35444
   1.07719
   1.40699
   1.96810
   1.99172
   1.09991
   1.75376
   1.11531
   1.79853
   1.83078
   1.90366
   1.34083
   1.22145
   1.24730
   1.58043
   1.92347
   1.08427
   1.26099
   1.92056
   1.86308
   0.74817
   1.93975
   1.69400
   1.80816
   1.50147
   0.68011
   1.49606
   1.13052
   1.44803
   1.81130
   1.11592
   0.98311
   1.66666
   1.39388
   1.10744
   1.0

   1.43372
   0.31994
   1.79909
   1.83068
   1.25757
   1.34003
   1.27312
   1.24656
   1.55463
   1.92215
   1.08563
   1.25533
   1.92012
   1.86262
   0.75080
   1.93402
   1.48816
   1.80913
   1.50137
   0.67163
   1.49648
   1.13113
   1.44807
   1.01132
   1.11928
   0.98231
   1.66576
   1.39313
   1.10755
   1.09219
   0.56570
   1.22685
   1.18170
   1.70438
   1.43862
   1.40242
   1.79219
   1.97345
   1.43320
   1.65624
   1.95150
   0.60747
   0.92429

ttb =    3.1930e+08
Optimizing qs: remain iter - 23
qs =

   1.116419
   1.329281
   1.495386
   1.649712
   1.515740
   1.953714
   1.270351
   1.062510
   1.763020
   1.659466
   1.430055
   0.705098
   1.066574
   1.608947
   1.487826
   1.851239
   1.353332
   1.750596
   1.359857
   0.545805
   1.721874
   1.601207
   1.693099
   1.109910
   1.641793
   1.360767
   1.268360
   1.207268
   1.687541
   1.332497
   0.320274
   1.354553
   1.074738
   1.133442
   1.969497
   1.991632
   1.102509
   1.433933
   0.324441


   1.273671
   1.242323
   1.546546
   1.922332
   1.084485
   1.223117
   1.920369
   1.862795
   0.740939
   1.925924
   1.459964
   1.006079
   1.499711
   0.676904
   1.496118
   1.129396
   1.446529
   1.008600
   1.116232
   0.898467
   1.663578
   1.393943
   1.108039
   1.090540
   0.558833
   1.226560
   1.181674
   1.703587
   1.438689
   1.403541
   1.792946
   1.920613
   1.430007
   1.656112
   1.934796
   0.078582
   0.928031

ttb =    2.2507e+07
Optimizing qs: remain iter - 15
qs =

   1.107218
   1.329314
   1.496113
   1.649789
   0.719585
   1.952424
   1.266493
   1.061308
   1.748711
   1.657637
   0.833031
   0.702213
   1.064015
   1.607384
   1.488594
   1.851238
   1.354302
   1.745820
   1.374313
   0.543180
   1.719126
   1.601239
   1.691330
   1.108572
   1.640476
   1.359943
   1.265412
   1.208160
   1.688308
   1.332498
   1.115005
   1.354537
   1.076343
   1.134231
   1.966269
   1.989393
   1.103800
   1.268177
   0.300335
   1.799071
   1.830943
   1.

   1.32402
   1.49656
   1.64902
   0.73642
   1.95175
   1.26628
   1.06091
   1.74880
   1.65730
   0.77996
   0.70825
   1.06376
   1.60706
   1.48641
   1.85199
   1.35434
   1.74504
   1.37394
   0.54888
   1.71893
   1.60092
   1.69264
   1.10964
   1.64050
   1.35971
   1.26575
   1.10303
   1.68810
   1.33276
   1.11889
   1.35437
   1.01930
   1.13381
   1.96651
   1.98823
   1.10608
   0.83635
   0.33594
   1.79917
   1.83032
   1.04682
   1.33780
   1.27490
   1.24276
   1.54053
   1.92432
   1.08655
   1.22369
   1.89978
   1.86272
   0.74077
   1.91752
   1.45980
   0.99852
   1.49917
   0.67809
   0.76061
   1.12933
   1.26021
   0.97358
   1.11815
   0.89744
   1.66344
   1.39469
   1.10796
   1.09016
   0.55922
   0.88485
   1.18160
   1.70320
   1.43860
   1.40399
   1.77893
   1.92061
   1.43027
   1.65616
   1.93469
   0.15616
   0.91534

ttb =    1.6006e+05
Optimizing qs: remain iter - 6
qs =

   1.10047
   1.32415
   1.18885
   1.64846
   0.74564
   1.95175
   1.26

In [9]:
qs
save('./res/terbcmenergydataqs', 'qs');
printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

qs =

   1.06706
   1.31970
   1.18849
   1.64838
   0.75705
   1.95161
   1.27104
   1.07063
   1.74866
   1.65691
   0.77992
   0.72346
   1.06272
   1.60915
   1.48821
   1.85199
   1.35491
   1.72129
   1.37482
   0.63724
   1.71880
   1.57231
   1.46633
   1.10931
   1.63999
   1.35945
   0.46737
   1.10689
   1.68840
   1.32764
   0.41787
   1.35438
   0.92481
   1.08177
   1.96691
   1.98815
   1.11190
   0.84126
   0.37149
   1.79947
   1.83249
   1.04919
   1.33693
   1.27494
   0.46101
   1.54672
   1.92523
   1.09590
   1.23001
   1.90203
   1.86226
   0.74434
   1.90899
   0.66721
   0.99905
   1.49966
   0.70709
   0.77434
   1.13836
   1.26115
   0.97714
   0.21543
   0.89555
   1.66335
   1.37388
   1.10812
   1.09062
   0.57151
   0.88115
   1.18158
   1.70293
   1.43917
   1.39137
   1.38625
   1.92060
   1.43032
   1.65616
   1.93528
   0.17516
   0.93237

TERBCM: SMSE 0.0002, MSLL 126.5329, NLPD 129.4084


In [ ]:
save('./res/terbcmenergydataqs_res', 'SMSE','MSLL','NLPD')

## 1.2 GRBCM vs TEGRBCM

### 1.2.1 GRBCM baseline

In [12]:
criterions = {'GRBCM'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing energy_data kmeans GRBCM
GRBCM: SMSE 0.0016, MSLL 5.4372, NLPD 8.3231


### 1.2.2 Grid q

In [13]:
criterions = {'TEGRBCM'}; %, 'TEGRBCM' 'TEGPoE'};
qs = 0.2:0.2:3.0;

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    for q = qs
        [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
        [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
        printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f - q %6.2f\r\n', criterion, SMSE,MSLL,NLPD,q);
        filename = sprintf('./res/tegrbcm%s_%s_%s_%6.2f.mat', dataset, partitionCriterion, criterion, q);
        save(filename, 'SMSE','MSLL','NLPD')
    endfor
endfor

Processing energy_data kmeans TEGRBCM
TEGRBCM: SMSE 0.0016, MSLL -1.0612, NLPD 1.8247 - q   0.20
TEGRBCM: SMSE 0.0016, MSLL -1.2763, NLPD 1.6096 - q   0.40
TEGRBCM: SMSE 0.0016, MSLL -0.5047, NLPD 2.3811 - q   0.60
TEGRBCM: SMSE 0.0016, MSLL 1.3873, NLPD 4.2731 - q   0.80
TEGRBCM: SMSE 0.0016, MSLL 5.4372, NLPD 8.3231 - q   1.00
TEGRBCM: SMSE 0.0016, MSLL 14.3929, NLPD 17.2787 - q   1.20
TEGRBCM: SMSE 0.0016, MSLL 35.6846, NLPD 38.5704 - q   1.40
TEGRBCM: SMSE 0.0016, MSLL 90.8930, NLPD 93.7788 - q   1.60
TEGRBCM: SMSE 0.0017, MSLL 247.6251, NLPD 250.5110 - q   1.80
TEGRBCM: SMSE 0.0017, MSLL 733.2615, NLPD 736.1473 - q   2.00
TEGRBCM: SMSE 0.0017, MSLL 2361.8222, NLPD 2364.7080 - q   2.20
TEGRBCM: SMSE 0.0017, MSLL 8203.6889, NLPD 8206.5747 - q   2.40
TEGRBCM: SMSE 0.0017, MSLL 30336.4468, NLPD 30339.3326 - q   2.60
TEGRBCM: SMSE 0.0017, MSLL 117856.0935, NLPD 117858.9794 - q   2.80
TEGRBCM: SMSE 0.0017, MSLL 475476.3111, NLPD 475479.1969 - q   3.00


### 1.2.3 Find single q

In [14]:
criterion = 'TEGRBCM';

In [15]:
printf('Optimizing q\r\n');
q = grad_q(models, criterion, opts, iter=50, bs=100, lr=0.001, lambda=10, init_q=1.5);

Optimizing q
Optimizing q: remain iter - 50, q - 1.5000
grad_q_norm =   -1.7290e+04
grad_q_reg = -234.19
Optimizing q: remain iter - 49, q - 1.4975
grad_q_norm = -68.071
grad_q_reg = -3.2211
Optimizing q: remain iter - 48, q - 1.4975
grad_q_norm = -1281.0
grad_q_reg =  0.15250
Optimizing q: remain iter - 47, q - 1.4973
grad_q_norm = -135.48
grad_q_reg = -0.15815
Optimizing q: remain iter - 46, q - 1.4973
grad_q_norm =   -1.6643e+05
grad_q_reg = -79.406
Optimizing q: remain iter - 45, q - 1.4762
grad_q_norm = -1060.4
grad_q_reg = -21.619
Optimizing q: remain iter - 44, q - 1.4760
grad_q_norm =   -4.6312e+04
grad_q_reg = -23.920
Optimizing q: remain iter - 43, q - 1.4673
grad_q_norm =   -3.2324e+04
grad_q_reg = -121.08
Optimizing q: remain iter - 42, q - 1.4631
grad_q_norm =   -1.8719e+05
grad_q_reg = -401.73
Optimizing q: remain iter - 41, q - 1.4389
grad_q_norm = -1393.1
grad_q_reg = -1.5468
Optimizing q: remain iter - 40, q - 1.4387
grad_q_norm =   -8.4224e+04
grad_q_reg = -64.593
Opt

In [16]:
[mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

### 1.2.4 Find multi q

In [17]:
printf('Optimizing q\r\n');
iqs = 1.01 + rand(opts.Ms,1);
qs = mul_grad_q(models, criterion, opts, iter=50, bs=100, lr=0.01, lambda=10, init_q=iqs);
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);

Optimizing q
Optimizing qs: remain iter - 50
qs =

   1.3352
   1.2170
   1.4822
   1.0162
   1.6526
   1.8487
   1.7002
   1.4597
   1.2379
   1.7434
   1.4762
   1.7069
   1.8983
   1.3787
   1.3958
   1.0473
   1.2090
   1.4166
   1.5364
   1.6222
   2.0097
   1.5230
   1.4505
   1.4185
   1.4207
   1.7239
   1.7025
   1.7372
   1.4054
   1.8481
   1.8734
   1.5668
   1.1874
   1.7273
   1.3418
   1.0417
   1.2157
   1.1384
   1.9755
   1.7262
   1.5113
   1.4041
   1.6600
   1.2677
   1.1991
   1.5560
   1.8303
   1.3286
   1.6746
   1.1095
   1.1867
   1.2046
   1.5639
   1.0270
   1.9852
   1.4700
   1.0624
   1.9169
   1.2641
   1.6760
   1.2535
   1.0331
   1.6076
   1.1334
   2.0009
   1.3880
   1.4542
   1.6643
   1.5394
   1.4233
   1.2901
   1.8578
   1.3590
   1.6504
   1.0337
   1.4485
   1.2452
   1.0563
   1.0461
   1.2838

ttb =    4.2388e+13
Optimizing qs: remain iter - 49
qs =

   1.3352
   1.2170
   1.4822
   1.0162
   1.6526
   1.8487
   1.7002
   1.4597
   1.2379


   1.01171
   1.60765
   0.33262
   2.00113
   1.38807
   1.45334
   1.66383
   1.53337
   1.42330
   1.15256
   1.85779
   1.35904
   1.65062
   1.03366
   1.44769
   1.24509
   1.05679
   1.04625
   1.28379

ttb =    1.3410e+08
Optimizing qs: remain iter - 32
qs =

   1.33516
   1.21692
   1.48204
   1.01620
   1.53079
   1.84839
   1.70035
   1.45922
   1.23770
   1.74321
   1.47690
   0.86313
   1.87608
   1.37888
   1.26360
   0.55610
   1.20829
   1.40536
   1.53617
   0.87598
   2.00985
   1.52321
   1.44757
   1.21517
   1.41899
   1.72391
   1.70283
   1.72996
   1.40537
   1.84785
   1.87375
   1.56676
   1.18648
   1.72307
   1.34165
   1.03962
   1.20975
   1.13456
   1.91505
   1.72619
   1.51160
   1.39881
   0.86003
   1.26775
   1.19804
   1.55505
   1.82044
   1.32804
   1.67413
   1.10012
   1.18667
   1.20296
   1.52613
   1.02685
   1.81192
   1.47024
   1.06190
   1.91085
   1.14902
   1.67335
   1.24192
   1.01171
   1.60766
   0.33259
   2.00007
   1.38808
   1.4

   1.37947
   1.26439
   0.55567
   1.20821
   1.40543
   1.50675
   0.87498
   2.00994
   1.52422
   1.44416
   1.21434
   1.41951
   1.72458
   1.70402
   1.73199
   1.40648
   1.72546
   1.87584
   1.56688
   1.18706
   1.71658
   1.34162
   1.03945
   1.19675
   1.13521
   1.11513
   1.72724
   1.51263
   1.39910
   0.86008
   1.27166
   1.17843
   1.55416
   1.82249
   1.32798
   1.67418
   1.10012
   1.18766
   1.17891
   1.52684
   1.02701
   1.52194
   1.47100
   1.06236
   1.75701
   1.14917
   1.67427
   1.02681
   1.00956
   1.60791
   0.33456
   2.00292
   1.35812
   1.45322
   1.66146
   1.53519
   1.42370
   1.15239
   1.85793
   1.33528
   1.65216
   1.03353
   1.44757
   1.24509
   1.05722
   1.04648
   1.20824

ttb =  8107.1
Optimizing qs: remain iter - 14
qs =

   1.33516
   1.21703
   1.48337
   1.01627
   0.72826
   1.84917
   1.70202
   1.45973
   1.23758
   1.74298
   1.47688
   0.63917
   1.87654
   1.37947
   1.26439
   0.55567
   1.20821
   1.40542
   1.50675
 

In [18]:
qs
save('./res/tegrbcmenergydataqs', 'qs');
printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

qs =

   1.33516
   1.21699
   1.48352
   1.01613
   0.72951
   1.84925
   1.70242
   1.32009
   1.23736
   1.74264
   1.47690
   0.64400
   1.87662
   1.37921
   1.26465
   0.55562
   1.20820
   1.28204
   1.45227
   0.77538
   2.01044
   1.52381
   1.43682
   1.21011
   1.41929
   1.72438
   1.70457
   1.73272
   1.40701
   1.72572
   1.87552
   1.56724
   1.14975
   1.42978
   1.34156
   1.03917
   0.99998
   0.96271
   1.11410
   1.72724
   1.51167
   1.39922
   0.85963
   1.27174
   1.17708
   1.55289
   1.80854
   1.32611
   0.87341
   1.10007
   1.18774
   1.17812
   1.52408
   1.02703
   1.50514
   1.47117
   1.04798
   1.75520
   1.13026
   1.65574
   1.02660
   0.57322
   1.60811
   0.33502
   1.20334
   1.35840
   1.45225
   1.66085
   1.53527
   1.42371
   0.95352
   1.85818
   1.29727
   1.17577
   1.03333
   1.44735
   1.24502
   1.05718
   1.04641
   1.20272

TEGRBCM: SMSE 0.0016, MSLL 57.7691, NLPD 60.6550


In [19]:
save('./res/tegrbcmenergydataqs_res', 'SMSE','MSLL','NLPD')

## 1.3 GPoE vs TEGPoE

### 1.3.1 GPoE baseline

In [20]:
criterions = {'GPoE'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing energy_data kmeans GPoE
GPoE: SMSE 0.0021, MSLL 5.1776, NLPD 8.0635


### 1.3.2 Grid q

In [21]:
partitionCriterions = {'kmeans'}; % 'random', 'kmeans', 'knkmeans'
criterions = {'TEGPoE'}; %, 'TEGRBCM' 'TEGPoE'};
qs = 0.2:0.2:3.0;

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    for q = qs
        [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
        [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
        printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f - q %6.2f\r\n', criterion, SMSE,MSLL,NLPD,q);
        filename = sprintf('./res/tegpoe%s_%s_%s_%6.2f.mat', dataset, partitionCriterion, criterion, q);
        save(filename, 'SMSE','MSLL','NLPD')
    endfor
endfor

Processing energy_data kmeans TEGPoE
TEGPoE: SMSE 0.0027, MSLL 1.5047, NLPD 4.3906 - q   0.20
TEGPoE: SMSE 0.0025, MSLL 0.3695, NLPD 3.2553 - q   0.40
TEGPoE: SMSE 0.0024, MSLL 0.6562, NLPD 3.5420 - q   0.60
TEGPoE: SMSE 0.0022, MSLL 2.0382, NLPD 4.9241 - q   0.80
TEGPoE: SMSE 0.0021, MSLL 5.1776, NLPD 8.0635 - q   1.00
TEGPoE: SMSE 0.0020, MSLL 11.9305, NLPD 14.8164 - q   1.20
TEGPoE: SMSE 0.0020, MSLL 26.8956, NLPD 29.7814 - q   1.40
TEGPoE: SMSE 0.0019, MSLL 61.7002, NLPD 64.5860 - q   1.60
TEGPoE: SMSE 0.0019, MSLL 146.8064, NLPD 149.6923 - q   1.80
TEGPoE: SMSE 0.0018, MSLL 364.8962, NLPD 367.7821 - q   2.00
TEGPoE: SMSE 0.0018, MSLL 947.9026, NLPD 950.7885 - q   2.20
TEGPoE: SMSE 0.0018, MSLL 2566.6408, NLPD 2569.5267 - q   2.40
TEGPoE: SMSE 0.0018, MSLL 7217.8346, NLPD 7220.7205 - q   2.60
TEGPoE: SMSE 0.0018, MSLL 21009.1201, NLPD 21012.0060 - q   2.80
TEGPoE: SMSE 0.0018, MSLL 63115.8865, NLPD 63118.7723 - q   3.00


### 1.3.3 Find single q

In [22]:
criterion = 'TEGPoE';

In [23]:
printf('Optimizing q\r\n');
q = grad_q(models, criterion, opts, iter=150, bs=100, lr=0.001, lambda=10, init_q=1.5);

Optimizing q
Optimizing q: remain iter - 150, q - 1.5000
grad_q_norm =   -5.3753e+05
grad_q_reg = -504.69
Optimizing q: remain iter - 149, q - 1.4313
grad_q_norm = -779.44
grad_q_reg = -22.096
Optimizing q: remain iter - 148, q - 1.4312
grad_q_norm =   -3.6112e+04
grad_q_reg = -87.214
Optimizing q: remain iter - 147, q - 1.4265
grad_q_norm = -8500.2
grad_q_reg = -43.130
Optimizing q: remain iter - 146, q - 1.4254
grad_q_norm =   -1.3685e+06
grad_q_reg = -192.69
Optimizing q: remain iter - 145, q - 1.2519
grad_q_norm =   -2.2861e+04
grad_q_reg = -147.46
Optimizing q: remain iter - 144, q - 1.2488
grad_q_norm =   -2.9246e+04
grad_q_reg = -41.126
Optimizing q: remain iter - 143, q - 1.2451
grad_q_norm = -1717.0
grad_q_reg = -18.385
Optimizing q: remain iter - 142, q - 1.2448
grad_q_norm =   -1.5384e+04
grad_q_reg = -17.706
Optimizing q: remain iter - 141, q - 1.2429
grad_q_norm =   -8.5612e+04
grad_q_reg = -101.21
Optimizing q: remain iter - 140, q - 1.2319
grad_q_norm =  436.34
grad_q_re

In [24]:
[mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

### 1.3.4 Find multi q

In [31]:
printf('Optimizing q\r\n');
iqs = 1.01 + rand(opts.Ms,1);
qs = mul_grad_q(models, criterion, opts, iter=50, bs=100, lr=0.1, lambda=10, init_q=iqs);
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);

Optimizing q
Optimizing qs: remain iter - 50
qs =

   1.0185
   1.9818
   1.4839
   1.8489
   1.6000
   1.1713
   1.8594
   1.5090
   1.2677
   1.9582
   1.6186
   1.3369
   1.8398
   1.8431
   1.2497
   1.8355
   1.5832
   1.7509
   1.5858
   1.3508
   1.9436
   1.5078
   1.4998
   1.3474
   1.0410
   1.9775
   1.7482
   1.4429
   1.5553
   1.4450
   1.4545
   1.4381
   1.0119
   1.9584
   1.6988
   1.5919
   1.0817
   1.7357
   1.9144
   1.8031
   1.3042
   1.3497
   1.3968
   1.3083
   1.2784
   1.4020
   1.0503
   1.6113
   1.3391
   1.5819
   1.3051
   2.0036
   1.2772
   1.5151
   1.8908
   1.7806
   1.7605
   1.3440
   1.9676
   1.5577
   1.9882
   1.7106
   1.3093
   1.4920
   2.0001
   1.0568
   1.9268
   1.7905
   1.4341
   1.7221
   1.8715
   1.2427
   1.5204
   1.6042
   1.7486
   1.2215
   1.4028
   1.7297
   1.2636
   1.6833

ttb = 0
Optimizing qs: remain iter - 49
qs =

   1.0186
   1.9818
   1.4839
   1.8489
   1.6001
   1.1718
   1.8594
   1.5090
   1.2677
   1.9582
  

   1.92715
   1.74310
   1.38179
   1.72231
   1.87158
   0.44836
   1.51395
   1.58682
   1.74871
   1.22372
   1.40313
   1.72920
   0.46538
   0.88391

ttb = 0
Optimizing qs: remain iter - 32
qs =

   1.01972
   1.97835
   1.48590
   1.84958
   1.60099
   0.37768
   1.84999
   1.51109
   1.26878
   1.95856
   1.61871
   1.04095
   1.84053
   1.84514
   1.25071
   1.83608
   1.58368
   0.95081
   1.58876
   1.35900
   1.94382
   1.28365
   1.50711
   1.35318
   1.04399
   1.97865
   1.74962
   1.21315
   1.55535
   1.42160
   0.66349
   1.44065
   0.60900
   1.96002
   1.69962
   1.59193
   0.29187
   1.62022
   1.89933
   1.80499
   1.30830
   1.35225
   1.39774
   1.30791
   1.27807
   1.40467
   0.99952
   1.60338
   1.34379
   1.58474
   1.30717
   1.99898
   1.22055
   1.34580
   1.89230
   1.78378
   1.71439
   1.31118
   1.96953
   1.55027
   1.18326
   1.71577
   1.30966
   1.49217
   1.99521
   1.06123
   1.92715
   1.74326
   1.38200
   1.72231
   1.87159
   0.44890
   1.51

   1.89967
   1.80576
   1.30454
   1.31044
   1.39789
   1.30749
   1.08904
   1.39720
   1.00187
   1.60319
   1.34919
   1.58747
   1.30900
   1.99190
   1.17992
   1.33997
   1.06732
   1.78462
   1.71232
   1.31406
   1.97100
   1.55220
   1.18150
   1.71161
   1.31004
   1.49336
   0.52430
   1.05874
   1.92790
   1.74197
   1.35498
   1.72247
   1.87222
   0.46170
   1.51624
   1.52002
   0.94872
   1.22728
   1.40499
   1.72989
   0.47349
   0.89869

ttb = 0
Optimizing qs: remain iter - 14
qs =

   0.98061
   1.92760
   1.48289
   1.85020
   1.59299
   0.38656
   1.85064
   1.50286
   1.26832
   1.95431
   0.81872
   1.04528
   1.84107
   1.81661
   1.16834
   1.83750
   1.58440
   0.95445
   1.50369
   1.34377
   1.94011
   1.28704
   1.44331
   1.35156
   1.04613
   1.97957
   1.75196
   1.14875
   1.55487
   1.36859
   0.68974
   1.44152
   0.49128
   1.72954
   1.70068
   1.59195
   0.44373
   1.59414
   1.89968
   1.80597
   1.30454
   1.31044
   1.39789
   1.30749
   1.08

In [32]:
qs
save('./res/tegpoeenergydataqs', 'qs');
printf('%s: SMSE %6.4f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

qs =

   0.1818562
   0.9612971
  -0.1149563
   1.0505042
   0.7708911
  -0.3444615
   1.0511992
   0.6999020
   0.4691516
   1.1545992
   0.0187512
   0.1673394
   1.0404141
   1.0175616
   0.3700139
   1.0375209
   0.7850534
   0.1600539
   0.7051139
   0.5481437
   1.1401700
   0.4804812
   0.6473983
   0.5275153
   0.2498562
   1.1801261
   0.9530360
   0.3508361
   0.7562035
   0.5700040
  -0.1007623
   0.6418408
  -0.2838549
   0.9305460
   0.9013883
   0.7919746
  -0.2409109
   0.7836512
   1.1008463
   2.6076354
   0.5061812
   0.5119934
  -0.1995030
  -0.2842082
   0.2908752
   0.5901761
   0.2056310
   0.8046867
   0.5514674
  -0.0018422
   0.5103075
   1.1928612
   0.2138234
   0.5417108
   0.2698102
   0.9858392
   0.1682279
  -0.1595204
   0.3742721
   0.7531940
   0.3852687
   0.9122772
   0.5100947
   0.6937696
  -0.2519278
   0.2558386
   1.1284759
   0.9436188
   0.5551844
   0.9230478
   1.0722441
  -0.3329245
   0.5748351
   0.7071842
   0.1488084
   0.4292052
   0.6

In [30]:
save('./res/tegpoeenergydataqs_res', 'SMSE','MSLL','NLPD')